# **ECR - Engenharia**

### Importação de bibliotecas


In [1]:
import pandas as pd
import numpy as np
import re
import os
import pytesseract
from PIL import Image
from datetime import datetime
import requests
import json
from tabulate import tabulate
from openpyxl import Workbook


#### **Chaves de acesso**

In [2]:
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

#### Enetendimento da Planilhas (Colunas/Linhas)

In [3]:
arquivo = 'APIS OMIE ECR.xlsx'

# Lista todas as abas da planilha
xls = pd.ExcelFile(arquivo)
print("Abas encontradas:", xls.sheet_names)

# Carrega a primeira aba (ou troque pelo nome desejado)
df = pd.read_excel(xls, sheet_name=0)

# Mostra as primeiras linhas para entender os dados
print("\nPrimeiras linhas:")
print(df.head())

# Informações gerais
print("\nInformações do DataFrame:")
print(df.info())

# Colunas disponíveis
print("\nColunas disponíveis:")
print(df.columns)

Abas encontradas: ['Planilha1']

Primeiras linhas:
                                               Geral  \
0  Alguns cadastros no Omie são compartilhados po...   
1       Clientes, Fornecedores, Transportadoras, etc   
2                         Clientes - Características   
3                                               Tags   
4                                           Projetos   

                                          Unnamed: 1 Unnamed: 2  \
0                                                NaN        NaN   
1  Cria/edita/consulta o cadastro de clientes, fo...         v1   
2    Cria/edita/consulta características de clientes         v1   
3  Cria/edita/consulta tags quem são usadas no ca...         v1   
4                       Cria/edita/consulta projetos         v1   

   DADO UTILIZADO NO GERENCIAL? Unnamed: 4  
0                           NaN        NaN  
1                           NaN         OK  
2                           NaN         OK  
3                           N

#### Filtro de endpoints

In [4]:
df.columns = ['Geral', 'Descricao', 'Versao', 'UsadoGerencial', 'Status']

# Filtrar linhas que possuem descrição e versão preenchidas
filtro_endpoints = df[ df['Versao'].notna()]

# Exibir os primeiros endpoints filtrados
filtro_endpoints.reset_index(drop=True)

,Geral,Descricao,Versao,UsadoGerencial,Status
0,"Clientes, Fornecedores, Transportadoras, etc","Cria/edita/consulta o cadastro de clientes, fo...",v1,NaN,OK
1,Clientes - Características,Cria/edita/consulta características de clientes,v1,NaN,OK
2,Tags,Cria/edita/consulta tags quem são usadas no ca...,v1,NaN,OK
3,Projetos,Cria/edita/consulta projetos,v1,NaN,OK
4,Empresas,Lista o cadastro da empresa,v1,NaN,OK
...,...,...,...,...,...
138,Etapas de Faturamento,Lista as etapas do faturamento,v1,NaN,OK
139,Tipo de utilização,Lista os Tipos de utilização,v1,NaN,OK
140,Classificação do Serviço,Lista as Classificações do Serviço,v1,NaN,OK
141,Documentos Fiscais,Listagem dos XMLs de Documentos Fiscais (NF-e/...,v1,NaN,OK


#### **Extração teste**

In [5]:
endpoint = "clientes"
versao = "v1"

base_url = f"https://app.omie.com.br/api/v1/geral/{endpoint}/"


headers = {
    'Content-Type': 'application/json'
}


body = {
    'call': 'ListarClientes',
    'app_key': APP_KEY,
    'app_secret': APP_SECRET,
    'param': [
        {
            "pagina": 1,
            "registros_por_pagina": 50
        }
    ]
}


response = requests.post(base_url, headers=headers, data=json.dumps(body))

if response.status_code == 200:
    print("Dados extraídos com sucesso!")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)


Dados extraídos com sucesso!
{
  "pagina": 1,
  "total_de_paginas": 60,
  "registros": 50,
  "total_de_registros": 2981,
  "clientes_cadastro": [
    {
      "bairro": "",
      "bloquear_faturamento": "N",
      "cep": "",
      "cidade": "",
      "cidade_ibge": "",
      "cnpj_cpf": "07.575.651/0001-59",
      "codigo_cliente_integracao": "",
      "codigo_cliente_omie": 679084426,
      "codigo_pais": "1058",
      "complemento": "",
      "dadosBancarios": {
        "agencia": "",
        "cChavePix": "",
        "codigo_banco": "",
        "conta_corrente": "",
        "doc_titular": "",
        "nome_titular": "",
        "transf_padrao": "N"
      },
      "endereco": "",
      "enderecoEntrega": {},
      "endereco_numero": "",
      "enviar_anexos": "N",
      "estado": "",
      "exterior": "N",
      "inativo": "N",
      "info": {
        "cImpAPI": "N",
        "dAlt": "07/01/2019",
        "dInc": "07/01/2019",
        "hAlt": "12:09:27",
        "hInc": "12:09:27",
    

In [6]:
endpoint = "projetos"

base_url = f"https://app.omie.com.br/api/v1/geral/projetos/" 

headers = {
    'Content-Type': 'application/json'
}

body = {
    "call": 'ListarProjetos', 
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50,
            "apenas_importado_api": "N"
        }
    ]
}


try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))

    # Check if the request was successful
    if response.status_code == 200:
        print("Dados extraídos com sucesso!")
        # Attempt to parse the JSON response
        try:
            print(json.dumps(response.json(), indent=2, ensure_ascii=False)) # Added ensure_ascii=False for better display of special characters
        except json.JSONDecodeError:
            print("Não foi possível decodificar a resposta JSON:")
            print(response.text)
    elif response.status_code == 401:
        print(f"Erro de autenticação (Unauthorized): {response.status_code}")
        print("Verifique se suas credenciais (APP_KEY e APP_SECRET) estão corretas e ativas.")
        print("Detalhes:", response.text)
    elif response.status_code == 500:
        print(f"Erro interno do servidor Omie: {response.status_code}")
        print("Detalhes:", response.text)
        print("Consulte a documentação da API Omie para o faultstring/faultcode específico.")
        # Example of how you might parse Omie's specific error structure if it's JSON
        try:
            error_details = response.json()
            if "faultstring" in error_details:
                print(f"Faultstring: {error_details['faultstring']}")
            if "faultcode" in error_details:
                print(f"Faultcode: {error_details['faultcode']}")
        except json.JSONDecodeError:
            pass # Already printed the raw text
    else:
        print(f"Erro na requisição: {response.status_code}")
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"Erro na conexão com a API: {e}")



Dados extraídos com sucesso!
{
  "pagina": 1,
  "total_de_paginas": 1,
  "registros": 14,
  "total_de_registros": 14,
  "cadastro": [
    {
      "codInt": "",
      "codigo": 679094336,
      "inativo": "N",
      "info": {
        "data_alt": "01/04/2019",
        "data_inc": "07/01/2019",
        "hora_alt": "11:58:14",
        "hora_inc": "12:29:24",
        "user_alt": "P000090237",
        "user_inc": "P000090237"
      },
      "nome": "OVER HEAD CORPORATIVO"
    },
    {
      "codInt": "",
      "codigo": 679099556,
      "inativo": "N",
      "info": {
        "data_alt": "07/01/2019",
        "data_inc": "07/01/2019",
        "hora_alt": "15:28:02",
        "hora_inc": "12:40:10",
        "user_alt": "P000102075",
        "user_inc": "P000090237"
      },
      "nome": "PROJETOS"
    },
    {
      "codInt": "",
      "codigo": 679099741,
      "inativo": "N",
      "info": {
        "data_alt": "07/01/2019",
        "data_inc": "07/01/2019",
        "hora_alt": "15:28:18",


In [12]:

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE EMPRESAS ---
endpoint_name = "Empresas" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"      # Módulo para Empresas na Omie API, conforme sua URL
api_path = "empresas"      # Parte final da URL do endpoint
call_method = "ListarEmpresas" # Nome do método de chamada da API
# Chave que contém a lista de empresas na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'empresa_cadastro' ou 'empresas' ou 'listaRegistros'
json_data_keys_to_try = ["empresa_cadastro", "empresas", "listaRegistros"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Empresas
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 100, # Você especificou 100 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_empresas = pd.DataFrame() # DataFrame vazio para armazenar os dados de empresas

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de empresas da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_empresas = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_empresas.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_empresas.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_empresas.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_empresas.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Empresas concluída! ---")

# Agora você tem o DataFrame 'df_empresas' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_empresas.to_csv('empresas_omie.csv', index=False, encoding='utf-8')
# df_empresas.to_excel('empresas_omie.xlsx', index=False)

Iniciando extração de dados para: Empresas
URL da Requisição: https://app.omie.com.br/api/v1/geral/empresas/
Método da Requisição (call): ListarEmpresas
--------------------------------------------------
Dados de Empresas extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Empresas' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 1,
  "registros": 1,
  "total_de_registros": 1,
  "empresas_cadastro": [
    {
      "bairro": "CERQUEIRA CESAR",
      "cep": "01415-002",
      "cidade": "SAO PAULO (SP)",
      "cnae": "7112000",
      "cnae_municipal": "",
      "cnpj": "42.161.372/0001-40",
      "codigo_empresa": "671524551",
      "codigo_pais": "1058",
      "complemento": "9 ANDAR",
      "csc_homologacao": "",
      "csc_id_homologacao": "",
      "csc_id_producao": "",
      "csc_producao": "",
      "ecd_codigo_cadastral": "",
      "ecd_codigo_instituicao_responsavel": "",
      "efd_atividade_i

In [15]:

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE DEPARTAMENTOS ---
endpoint_name = "Departamentos" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"           # Módulo para Departamentos na Omie API, conforme sua URL
api_path = "departamentos"      # Parte final da URL do endpoint
call_method = "ListarDepartamentos" # Nome do método de chamada da API
# Chave que contém a lista de departamentos na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'departamentos_cadastro', 'listaDepartamentos', 'listaRegistros' ou 'departamentos'
json_data_keys_to_try = ["departamentos_cadastro", "listaDepartamentos", "listaRegistros", "departamentos"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Departamentos
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50 # Você especificou 50 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_departamentos = pd.DataFrame() # DataFrame vazio para armazenar os dados de departamentos

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de departamentos da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_departamentos = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_departamentos.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_departamentos.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_departamentos.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_departamentos.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Departamentos concluída! ---")

# Agora você tem o DataFrame 'df_departamentos' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_departamentos.to_csv('departamentos_omie.csv', index=False, encoding='utf-8')
# df_departamentos.to_excel('departamentos_omie.xlsx', index=False)

Iniciando extração de dados para: Departamentos
URL da Requisição: https://app.omie.com.br/api/v1/geral/departamentos/
Método da Requisição (call): ListarDepartamentos
--------------------------------------------------
Dados de Departamentos extraídos com sucesso!
Chave de dados JSON identificada: 'departamentos'

--- Análise Exploratória para Departamentos ---
Shape do DataFrame: (50, 4)

Primeiras 5 linhas:
      codigo                     descricao    estrutura inativo
0  671524591        OVER HEAD COORPORATIVO      001.001       N
1  671524592                    SUPERVISÃO      001.002       N
2  671524593              ECR -CONSOLIDADO          001       N
3  671524594                      PROJETOS      001.003       N
4  679097416  120_ECR/EURO ESTUDIOS -BID 3  001.002.001       S

Informações do DataFrame (tipos de dados e contagem de não-nulos):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column     Non-Null Count  D

In [16]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CATEGORIAS ---
endpoint_name = "Categorias" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"        # Módulo para Categorias na Omie API, conforme sua URL
api_path = "categorias"      # Parte final da URL do endpoint
call_method = "ListarCategorias" # Nome do método de chamada da API
# Chave que contém a lista de categorias na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'categorias_cadastro', 'listaCategorias', 'listaRegistros' ou 'categorias'
json_data_keys_to_try = ["categorias_cadastro", "listaCategorias", "listaRegistros", "categorias"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Categorias
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50 # Você especificou 50 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_categorias = pd.DataFrame() # DataFrame vazio para armazenar os dados de categorias

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de categorias da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_categorias = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_categorias.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_categorias.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_categorias.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_categorias.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Categorias concluída! ---")

# Agora você tem o DataFrame 'df_categorias' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_categorias.to_csv('categorias_omie.csv', index=False, encoding='utf-8')
# df_categorias.to_excel('categorias_omie.xlsx', index=False)

Iniciando extração de dados para: Categorias
URL da Requisição: https://app.omie.com.br/api/v1/geral/categorias/
Método da Requisição (call): ListarCategorias
--------------------------------------------------
Dados de Categorias extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Categorias' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 6,
  "registros": 50,
  "total_de_registros": 256,
  "categoria_cadastro": [
    {
      "categoria_superior": "0",
      "codigo": "0.01",
      "codigo_dre": "",
      "conta_despesa": "N",
      "conta_inativa": "N",
      "conta_receita": "N",
      "dadosDRE": {},
      "definida_pelo_usuario": "N",
      "descricao": "Transferência",
      "descricao_padrao": "Transferência",
      "id_conta_contabil": "",
      "nao_exibir": "S",
      "natureza": "",
      "tag_conta_contabil": "",
      "tipo_categoria": "",
      "totalizadora": "S",
      "transferencia"

In [17]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CONTAS CORRENTES ---
endpoint_name = "Contas Correntes" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"            # Módulo para Contas Correntes na Omie API, conforme sua URL
api_path = "contacorrente"       # Parte final da URL do endpoint
call_method = "ListarContasCorrentes" # Nome do método de chamada da API
# Chave que contém a lista de contas correntes na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'contaCorrenteCadastro', 'listaContasCorrentes', 'listaRegistros' ou 'contasCorrentes'
json_data_keys_to_try = ["contaCorrenteCadastro", "listaContasCorrentes", "listaRegistros", "contasCorrentes"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Contas Correntes
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 100, # Você especificou 100 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_contas_correntes = pd.DataFrame() # DataFrame vazio para armazenar os dados de contas correntes

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contas correntes da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_contas_correntes = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_contas_correntes.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_contas_correntes.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_contas_correntes.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_contas_correntes.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Contas Correntes concluída! ---")

# Agora você tem o DataFrame 'df_contas_correntes' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_contas_correntes.to_csv('contas_correntes_omie.csv', index=False, encoding='utf-8')
# df_contas_correntes.to_excel('contas_correntes_omie.xlsx', index=False)

Iniciando extração de dados para: Contas Correntes
URL da Requisição: https://app.omie.com.br/api/v1/geral/contacorrente/
Método da Requisição (call): ListarContasCorrentes
--------------------------------------------------
Dados de Contas Correntes extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Contas Correntes' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 1,
  "registros": 63,
  "total_de_registros": 63,
  "ListarContasCorrentes": [
    {
      "bairro": "",
      "bloqueado": "N",
      "bol_instr1": "Sr. Caixa, receber até 10 dias após o vencimento.",
      "bol_instr2": "",
      "bol_instr3": "",
      "bol_instr4": "",
      "bol_sn": "N",
      "cCodCCInt": "",
      "cEstabelecimento": "",
      "cTipoCartao": "",
      "cancinstr": "",
      "cep": "",
      "cidade": "",
      "cnab_esp": "",
      "cobr_esp": "",
      "cobr_sn": "N",
      "codigo_agencia": "6613",
      "codigo_

In [18]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CONTAS A PAGAR ---
endpoint_name = "Contas a Pagar" # Nome da categoria para exibição e uso no DataFrame
base_module = "financas"         # Módulo para Contas a Pagar na Omie API, conforme sua URL
api_path = "contapagar"          # Parte final da URL do endpoint
call_method = "ListarContasPagar" # Nome do método de chamada da API
# Chave que contém a lista de contas a pagar na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'titulosPagar', 'listaContasPagar', 'listaRegistros' ou 'contasPagar'
json_data_keys_to_try = ["titulosPagar", "listaContasPagar", "listaRegistros", "contasPagar"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Contas a Pagar
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 20, # Você especificou 20 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_contas_pagar = pd.DataFrame() # DataFrame vazio para armazenar os dados de contas a pagar

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contas a pagar da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_contas_pagar = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_contas_pagar.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_contas_pagar.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_contas_pagar.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_contas_pagar.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Contas a Pagar concluída! ---")

# Agora você tem o DataFrame 'df_contas_pagar' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_contas_pagar.to_csv('contas_pagar_omie.csv', index=False, encoding='utf-8')
# df_contas_pagar.to_excel('contas_pagar_omie.xlsx', index=False)

Iniciando extração de dados para: Contas a Pagar
URL da Requisição: https://app.omie.com.br/api/v1/financas/contapagar/
Método da Requisição (call): ListarContasPagar
--------------------------------------------------
Dados de Contas a Pagar extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Contas a Pagar' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 2183,
  "registros": 20,
  "total_de_registros": 43642,
  "conta_pagar_cadastro": [
    {
      "baixa_bloqueada": "N",
      "bloqueado": "N",
      "categorias": [
        {
          "codigo_categoria": "2.11.98",
          "percentual": 100,
          "valor": 1800
        }
      ],
      "codigo_categoria": "2.11.98",
      "codigo_cliente_fornecedor": 679084500,
      "codigo_lancamento_integracao": "",
      "codigo_lancamento_omie": 679094339,
      "codigo_projeto": 679094336,
      "codigo_tipo_documento": "99999",
      "data_emissao": "

In [19]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE PLANO DE CONTAS (DRE) ---
endpoint_name = "Plano de Contas (DRE)" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"                  # Módulo para DRE na Omie API, conforme sua URL
api_path = "dre"                       # Parte final da URL do endpoint
call_method = "ListarCadastroDRE"      # Nome do método de chamada da API
# Chave que contém a lista de contas DRE na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'listaContas', 'listaRegistros' ou 'contasDRE'
json_data_keys_to_try = ["listaContas", "listaRegistros", "contasDRE"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Plano de Contas (DRE)
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "apenasContasAtivas": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_plano_contas_dre = pd.DataFrame() # DataFrame vazio para armazenar os dados do plano de contas

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contas DRE da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_plano_contas_dre = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_plano_contas_dre.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_plano_contas_dre.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_plano_contas_dre.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_plano_contas_dre.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Plano de Contas (DRE) concluída! ---")

# Agora você tem o DataFrame 'df_plano_contas_dre' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_plano_contas_dre.to_csv('plano_contas_dre_omie.csv', index=False, encoding='utf-8')
# df_plano_contas_dre.to_excel('plano_contas_dre_omie.xlsx', index=False)

Iniciando extração de dados para: Plano de Contas (DRE)
URL da Requisição: https://app.omie.com.br/api/v1/geral/dre/
Método da Requisição (call): ListarCadastroDRE
--------------------------------------------------
Dados de Plano de Contas (DRE) extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Plano de Contas (DRE)' na resposta da API.
Resposta completa da API (para depuração): {
  "totalRegistros": 28,
  "dreLista": [
    {
      "codigoDRE": "1",
      "descricaoDRE": "Lucro Bruto",
      "naoExibirDRE": "N",
      "nivelDRE": 1,
      "sinalDRE": "",
      "totalizaDRE": "S"
    },
    {
      "codigoDRE": "1.01",
      "descricaoDRE": "Receita Líquida Operacional",
      "naoExibirDRE": "N",
      "nivelDRE": 2,
      "sinalDRE": "",
      "totalizaDRE": "S"
    },
    {
      "codigoDRE": "1.01.01",
      "descricaoDRE": "Receita Bruta de Vendas",
      "naoExibirDRE": "N",
      "nivelDRE": 3,
      "sinalDRE": "+",
      "totalizaDRE": "N"
   

In [20]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE NF-e (CONSULTAS) ---
endpoint_name = "NF-e (Consultas)" # Nome da categoria para exibição e uso no DataFrame
base_module = "produtos"         # Módulo para NF-e na Omie API, conforme sua URL
api_path = "nfconsultar"         # Parte final da URL do endpoint
call_method = "ListarNF"         # Nome do método de chamada da API
# Chave que contém a lista de NF-e na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'nf_cadastro', 'listaNF', 'notasFiscais', 'listaRegistros' ou 'nfList'
json_data_keys_to_try = ["nf_cadastro", "listaNF", "notasFiscais", "listaRegistros", "nfList"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para NF-e (Consultas)
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 20, # Você especificou 20 aqui
            "apenas_importado_api": "N",
            "ordenar_por": "CODIGO"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_nfe_consultas = pd.DataFrame() # DataFrame vazio para armazenar os dados de NF-e

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de NF-e da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_nfe_consultas = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_nfe_consultas.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_nfe_consultas.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_nfe_consultas.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_nfe_consultas.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de NF-e (Consultas) concluída! ---")

# Agora você tem o DataFrame 'df_nfe_consultas' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_nfe_consultas.to_csv('nfe_consultas_omie.csv', index=False, encoding='utf-8')
# df_nfe_consultas.to_excel('nfe_consultas_omie.xlsx', index=False)

Iniciando extração de dados para: NF-e (Consultas)
URL da Requisição: https://app.omie.com.br/api/v1/produtos/nfconsultar/
Método da Requisição (call): ListarNF
--------------------------------------------------
Dados de NF-e (Consultas) extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'NF-e (Consultas)' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 103,
  "registros": 20,
  "total_de_registros": 2060,
  "nfCadastro": [
    {
      "compl": {
        "cChaveNFe": "35181243283811013995550010001945931286166664",
        "cCodCateg": "2.04.06",
        "cModFrete": "9",
        "nIdNF": 679597940,
        "nIdPedido": 0,
        "nIdReceb": 679596110,
        "nIdTransp": 0
      },
      "det": [
        {
          "nfProdInt": {
            "cCodItemInt": "",
            "cCodProdInt": "",
            "nCodItem": 679597944,
            "nCodProd": 679597943
          },
          "prod": {
      

In [21]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE SERVIÇOS ---
endpoint_name = "Serviços" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"   # Módulo para Serviços na Omie API, conforme sua URL
api_path = "servico"       # Parte final da URL do endpoint
call_method = "ListarCadastroServico" # Nome do método de chamada da API
# Chave que contém a lista de serviços na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'servicoCadastro', 'listaServicos', 'listaRegistros' ou 'servicos'
json_data_keys_to_try = ["servicoCadastro", "listaServicos", "listaRegistros", "servicos"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Serviços
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "nPagina": 1,
            "nRegPorPagina": 20 # Você especificou 20 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_servicos = pd.DataFrame() # DataFrame vazio para armazenar os dados de serviços

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de serviços da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_servicos = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_servicos.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_servicos.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_servicos.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_servicos.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Serviços concluída! ---")

# Agora você tem o DataFrame 'df_servicos' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_servicos.to_csv('servicos_omie.csv', index=False, encoding='utf-8')
# df_servicos.to_excel('servicos_omie.xlsx', index=False)

Iniciando extração de dados para: Serviços
URL da Requisição: https://app.omie.com.br/api/v1/servicos/servico/
Método da Requisição (call): ListarCadastroServico
--------------------------------------------------
Dados de Serviços extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Serviços' na resposta da API.
Resposta completa da API (para depuração): {
  "nPagina": 1,
  "nTotPaginas": 1,
  "nRegistros": 3,
  "nTotRegistros": 3,
  "cadastros": [
    {
      "cabecalho": {
        "cCodCateg": "",
        "cCodLC116": "7.17",
        "cCodServMun": "01805",
        "cCodigo": "SRV00001",
        "cDescricao": "Acompanhamento e fiscalização",
        "cIdTrib": "01",
        "cTipoDesc": "",
        "nAliqDesc": 0,
        "nIdNBS": "",
        "nPrecoUnit": 0,
        "nValorDesc": 0
      },
      "descricao": {
        "cDescrCompleta": "DADOS PARA DEPÓSITO||BANCO DO BRASIL|AGÊNCIA 0303-4|CONTA 106.745-1"
      },
      "impostos": {
        "cRetCOF

In [22]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE ORDENS DE SERVIÇO ---
endpoint_name = "Ordens de Serviço" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"             # Módulo para Ordens de Serviço na Omie API, conforme sua URL
api_path = "os"                      # Parte final da URL do endpoint
call_method = "ListarOS"             # Nome do método de chamada da API
# Chave que contém a lista de ordens de serviço na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'osCadastro', 'listaOS', 'listaRegistros' ou 'ordensServico'
json_data_keys_to_try = ["osCadastro", "listaOS", "listaRegistros", "ordensServico"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Ordens de Serviço
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50, # Você especificou 50 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_ordens_servico = pd.DataFrame() # DataFrame vazio para armazenar os dados de ordens de serviço

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de ordens de serviço da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_ordens_servico = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_ordens_servico.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_ordens_servico.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_ordens_servico.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_ordens_servico.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Ordens de Serviço concluída! ---")

# Agora você tem o DataFrame 'df_ordens_servico' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_ordens_servico.to_csv('ordens_servico_omie.csv', index=False, encoding='utf-8')
# df_ordens_servico.to_excel('ordens_servico_omie.xlsx', index=False)

Iniciando extração de dados para: Ordens de Serviço
URL da Requisição: https://app.omie.com.br/api/v1/servicos/os/
Método da Requisição (call): ListarOS
--------------------------------------------------
Dados de Ordens de Serviço extraídos com sucesso!
Chave de dados JSON identificada: 'osCadastro'

--- Análise Exploratória para Ordens de Serviço ---
Shape do DataFrame: (50, 8)

Primeiras 5 linhas:
                                           Cabecalho  \
0  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
1  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
2  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
3  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
4  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   

                                       Departamentos  \
0  [{'cCodDepto': '7280778563', 'nPerc': 100, 'nV...   
1  [{'cCodDepto': '7280778563', 'nPerc': 100, 'nV...   
2  [{'cCodDepto': '7280778563', 'nPerc': 100, 'nV...   
3  [{'cCodDepto': '7280778563', 'nPe

In [23]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CONTRATOS DE SERVIÇO ---
endpoint_name = "Contratos de Serviço" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"               # Módulo para Contratos de Serviço na Omie API, conforme sua URL
api_path = "contrato"                  # Parte final da URL do endpoint
call_method = "ListarContratos"        # Nome do método de chamada da API
# Chave que contém a lista de contratos de serviço na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'contratoCadastro', 'listaContratos', 'listaRegistros' ou 'contratos'
json_data_keys_to_try = ["contratoCadastro", "listaContratos", "listaRegistros", "contratos"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Contratos de Serviço
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50, # Você especificou 50 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_contratos_servico = pd.DataFrame() # DataFrame vazio para armazenar os dados de contratos de serviço

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contratos de serviço da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_contratos_servico = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_contratos_servico.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_contratos_servico.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_contratos_servico.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_contratos_servico.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Contratos de Serviço concluída! ---")

# Agora você tem o DataFrame 'df_contratos_servico' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_contratos_servico.to_csv('contratos_servico_omie.csv', index=False, encoding='utf-8')
# df_contratos_servico.to_excel('contratos_servico_omie.xlsx', index=False)

Iniciando extração de dados para: Contratos de Serviço
URL da Requisição: https://app.omie.com.br/api/v1/servicos/contrato/
Método da Requisição (call): ListarContratos
--------------------------------------------------
Dados de Contratos de Serviço extraídos com sucesso!
Chave de dados JSON identificada: 'contratoCadastro'

--- Análise Exploratória para Contratos de Serviço ---
Shape do DataFrame: (50, 7)

Primeiras 5 linhas:
                                           cabecalho  \
0  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
1  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
2  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
3  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
4  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   

                                       departamentos  \
0  [{'cCodDep': '7280778563', 'cDesDep': '', 'nPe...   
1  [{'cCodDep': '7280778563', 'cDesDep': '', 'nPe...   
2  [{'cCodDep': '7280778563', 'cDesDep': '', 'nPe...   
3  [{'cC

In [24]:
import requests
import json
import pandas as pd
import io

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE NFS-e (CONSULTAS) ---
endpoint_name = "NFS-e (Consultas)" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"             # Módulo para NFS-e na Omie API, conforme sua URL
api_path = "nfse"                    # Parte final da URL do endpoint
call_method = "ListarNFSEs"          # Nome do método de chamada da API
# Chave que contém a lista de NFS-e na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'nfseCadastro', 'listaNFSEs', 'notasServico', 'listaRegistros' ou 'nfseList'
json_data_keys_to_try = ["nfseCadastro", "listaNFSEs", "notasServico", "listaRegistros", "nfseList"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para NFS-e (Consultas)
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "nPagina": 1,
            "nRegPorPagina": 20 # Você especificou 20 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_nfse_consultas = pd.DataFrame() # DataFrame vazio para armazenar os dados de NFS-e

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de NFS-e da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_nfse_consultas = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_nfse_consultas.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_nfse_consultas.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_nfse_consultas.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_nfse_consultas.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de NFS-e (Consultas) concluída! ---")

# Agora você tem o DataFrame 'df_nfse_consultas' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_nfse_consultas.to_csv('nfse_consultas_omie.csv', index=False, encoding='utf-8')
# df_nfse_consultas.to_excel('nfse_consultas_omie.xlsx', index=False)

Iniciando extração de dados para: NFS-e (Consultas)
URL da Requisição: https://app.omie.com.br/api/v1/servicos/nfse/
Método da Requisição (call): ListarNFSEs
--------------------------------------------------
Dados de NFS-e (Consultas) extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'NFS-e (Consultas)' na resposta da API.
Resposta completa da API (para depuração): {
  "nPagina": 1,
  "nTotPaginas": 212,
  "nRegistros": 20,
  "nTotRegistros": 4223,
  "nfseEncontradas": [
    {
      "Adicionais": {
        "cCodigoCategoria": "1.01.02",
        "nCodigoCC": 679106865
      },
      "Alteracao": {
        "cDataAlteracao": "16/01/2019",
        "cHoraAlteracao": "10:31:20",
        "cUsuarioAlteracao": "P000102075"
      },
      "Cabecalho": {
        "cAmbienteNFSe": "P",
        "cCNPJDestinatario": "43.052.497/0001-02",
        "cCNPJEmissor": "42.161.372/0001-40",
        "cCidadeEmissor": "SAO PAULO (SP)",
        "cCodMunDestinatario": "3550308"